# 0.导包

In [1]:
import os
#数据处理
import pandas as pd
import numpy as np
import random
import sklearn.preprocessing as preprocessing
#可视化
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
#ML
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import (GradientBoostingClassifier, GradientBoostingRegressor, 
                              RandomForestClassifier, RandomForestRegressor)
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.model_selection import learning_curve

import warnings
warnings.filterwarnings('ignore')

# 1.数据载入

In [3]:
# 载入数据，需要5GB内存
feature_type = (np.int64,np.int64,(np.str,60),(np.str,1990),np.int64,np.int64,np.int8,np.int8,np.int8,np.int8,np.int64,np.int8,np.int64,np.int64,np.int64,np.int64,np.int64,np.int32,(np.str,385),np.int64,np.int8,float,np.int16,float,float,float,np.int8)
data = np.genfromtxt('data/round1_train.txt',dtype = feature_type,delimiter=' ',skip_header=1)
print(data.shape)
data = pd.DataFrame(data)

# 读入缺失值
data[data == -1] = np.nan
data[data == -1.00] = np.nan
data[data == '-1'] = np.nan

(18371,)


# 2.重复值处理

In [4]:
print(len(data))
data = data.drop_duplicates()
print(len(data))

18371
18371


# 3.缺失值处理

- 缺失的值比较少，取均值

- 缺失的值中间，对于类别信息的列可以将缺失作为新的类别做 one-hot

## 3.1缺失值
- 广告商品的品牌编号:丢弃缺失值所在样本
- 广告商品的城市编号:丢弃缺失值所在样本
- 广告商品的销量等级（数值越大表示销量越高）:


- 用户的预测性别编号：新建一个类别
- 用户的预测年龄等级（数值越大表示年龄越大）：
- 用户的预测职业编号：
- 用户的星级编号（越大星级越高）：
- 根据查询词预测的类目属性列表：丢弃缺失值所在样本


- 店铺的好评率（越大越高）：丢弃缺失值所在样本
- 店铺的服务态度评分（越大越高）:丢弃缺失值所在样本
- 店铺的物流服务评分（越大越高）:丢弃缺失值所在样本
- 店铺的描述相符评分（越大越高）:丢弃缺失值所在样本

## 3.2扔掉缺失值的样本
- 这样虽然信息用的不充分，但也不会引入额外的误差。

## 3.3用平均值或者中位数来填充缺失值
- 连续值：
    - 偏正态分布，使用均值代替，可以保持数据的均值
    - 偏长尾分布，使用中值代替，避免受 outlier 的影响
- 离散值：使用众数代替

## 3.4空与不空可以作为特征值（扩充取值域）

In [5]:
print(data.isnull().sum())
print(len(data))

f0       0
f1       0
f2       0
f3       0
f4      18
f5       6
f6       0
f7      35
f8       0
f9       0
f10      0
f11    469
f12     18
f13     18
f14     18
f15      0
f16      0
f17      0
f18      0
f19      0
f20      0
f21      0
f22      0
f23      1
f24      1
f25      1
dtype: int64
18371


In [14]:
data.dropna(axis = 0,subset = ['f4','f5','f18','f21','f23','f24','f25'],inplace = True)

In [7]:
print(data.isnull().sum())

f0      0
f1      0
f2      0
f3      0
f4     18
f5      6
f6      0
f7      0
f8      0
f9      0
f10     0
f11     0
f12     0
f13     0
f14     0
f15     0
f16     0
f17     0
f18     0
f19     0
f20     0
f21     0
f22     0
f23     1
f24     1
f25     1
dtype: int64


In [9]:
# 对于test数据
# data['f4'].fillna(int(data['f4'].mean()),inplace = True) # 广告商品的销量等级（数值越大表示销量越高）
# data['f5'].fillna(int(data['f6'].mean()),inplace = True) # 广告商品的销量等级（数值越大表示销量越高）

# data['f23'].fillna(data['f23'].mean(),inplace = True) # 广告商品的销量等级（数值越大表示销量越高）
# data['f24'].fillna(data['f24'].mean(),inplace = True) # 广告商品的销量等级（数值越大表示销量越高）
# data['f25'].fillna(data['f25'].mean(),inplace = True) # 广告商品的销量等级（数值越大表示销量越高）

In [6]:
data['f11'].fillna(3,inplace = True) # 用户的预测性别编号

data['f7'].fillna(int(data['f7'].mean()),inplace = True) # 广告商品的销量等级（数值越大表示销量越高）
data['f12'].fillna(int(data['f12'].mean()),inplace = True) # 广告商品被收藏次数的等级（数值越大表示被收藏次数越高）
data['f13'].fillna(int(data['f13'].mode()),inplace = True) # 用户的预测职业编号
data['f14'].fillna(int(data['f14'].median()),inplace = True) # 用户的星级编号（越大星级越高）

## 3.5利用其它的变量去估计缺失变量的值

In [ ]:
# 预测f7广告商品的销售等级
# 相关变量：f6、f8、f9、f20、f21、f22、f23、f24、f25、f16
# 暂时不用
def setMissingAges(df,train_labels,test_label):

    # 去模型变量
    age_df = df[test_label + train_labels]

    # 将年龄缺失样本挑出
    knownAge = age_df.loc[ (df[test_label].notnull()) ]
    unknownAge = age_df.loc[ (df[test_label].isnull()) ]

    # 取y值
    y = knownAge.values[:, 0]

    # 取X值
    X = knownAge.values[:, 1::]

    # 建模
    rtr = RandomForestRegressor(n_estimators=2000, n_jobs=-1)
    
    rtr.fit(X, y)

    # 预测
    predictedAges = rtr.predict(unknownAge.values[:, 1::])

    # 赋值
    df.loc[ (df[test_label].isnull()), test_label ] = predictedAges

    return df

setMissingAges(data,'f7',['f6','f8', 'f9', 'f20', 'f21','f22','f23','f24','f25','f26'])

In [17]:
print(data.isnull().sum())
print(len(data))

f0     0
f1     0
f2     0
f3     0
f4     0
f5     0
f6     0
f7     0
f8     0
f9     0
f10    0
f11    0
f12    0
f13    0
f14    0
f15    0
f16    0
f17    0
f18    0
f19    0
f20    0
f21    0
f22    0
f23    0
f24    0
f25    0
f26    0
dtype: int64
477303


# 4.离群点处理
- 散点图中：f6\f7\f9\f20\f21\f22\f23\f24\f25这几个特征貌似有离群点
- 离群点处理完了之后，再看一下变量分布是否符合正态

# 5.保存结果

In [18]:
data.to_csv("data/round1_train.csv",index=False,sep=" ")  